## Lab 1 - Analyzing Real Estate Data in St. Petersburg
We have data from Yandex.Realty classified https://realty.yandex.ru containing real estate listings for apartments in St. Petersburg and Leningrad Oblast from 2016 till the middle of August 2018. In this Lab you'll learn how to work with datasets in python using pandas library for data analysis using the provided data. 
For datasets like this it becomes hard to use Excel because of its size. If you learn how to work with pandas you'll be able to do basic analytical research much faster than in Excel.
To execute code IPython notebook click on the Cell and press Shift-Enter.
If you don't have any experience with Jupyter notebooks, please look at this tutorial: https://mybinder.org/v2/gh/ipython/ipython-in-depth/master?filepath=binder/Index.ipynb

### Main objectives
After successful completion of the lab work students will be able to:
-	Analyze large datasets that don't work well in Excel using IPython notebook and pandas library
-   Find outliers and remove them in the datasets
-	Calculate different statistics on data fields


### Tasks
-	Calculate median and mean sell and rent prices in St. Petersburg
-   Find outliers: too cheap or too expensive apartments which seem to be a wrong data. Remove them from the dataset.
-	Find houses with the most cheapest and most expensive prices per square meter.
-   Analyze how many rent offers have the commission and what is the most popular commission



In [1]:
# let's import pandas library and set options to be able to view data right in the browser
# Assign pd for a short alias for pandas library
import pandas as pd
# Call set_option function fron pandas library to edit them limit for number of the displayed
# columns, width of all columns and maximum width of one column
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
# let's look what is inside our data file to understand how we can export it. 
# We can use linux command "head" which shows the first row in the file. 
# In IPython notebooks you can execute linux commands using "!" prefixe before the command name. 
# This mechanism is called "IPython magic"
!head -n 3 spb.real.estate.archive.2018.tsv

offer_id	first_day_exposition	last_day_exposition	last_price	floor	open_plan	rooms	studio	area	kitchen_area	living_area	agent_fee	renovation	offer_type	category_type	unified_address	building_id
5490504238877583424	2014-11-27T00:00:00+03:00	2016-03-14T00:00:00+03:00	3676740.0	19	False	1	False	27.4	16.56	16.56		0.0	1	2	Россия, Санкт-Петербург, улица Шкапина, 9-11	3552413344225333068
5490504238903444021	2014-11-27T00:00:00+03:00	2016-04-28T00:00:00+03:00	7038880.0	2	False	3	False	85.84	19.64	41.2		0.0	1	2	Россия, Санкт-Петербург, Кушелевская дорога, 3к4	7274736308217971906


In [4]:
# if you don't remember what linux command does (i.e. what is -n), you can always look for help
!head --help

Usage: head [OPTION]... [FILE]...
Print the first 10 lines of each FILE to standard output.
With more than one FILE, precede each with a header giving the file name.

With no FILE, or when FILE is -, read standard input.

Mandatory arguments to long options are mandatory for short options too.
  -c, --bytes=[-]NUM       print the first NUM bytes of each file;
                             with the leading '-', print all but the last
                             NUM bytes of each file
  -n, --lines=[-]NUM       print the first NUM lines instead of the first 10;
                             with the leading '-', print all but the last
                             NUM lines of each file
  -q, --quiet, --silent    never print headers giving file names
  -v, --verbose            always print headers giving file names
  -z, --zero-terminated    line delimiter is NUL, not newline
      --help     display this help and exit
      --version  output version information and exit

NUM may have a mu

In [5]:
!ls

data					lab2_building_model.ipynb
lab1_1_EDA_real_estate_data_done.ipynb	lectures
lab1_1_EDA_real_estate_data.ipynb	spb.real.estate.archive.2018.tsv
lab1_2_visualization_done.ipynb		spb.real.estate.archive.sample5000.tsv
lab1_2_visualization.ipynb		Untitled.ipynb


In [6]:
# we see that it's tab separated file, we can load it with pd.read_table method
# load our dataset and see which data it contains.
# HINT: you can type 'Tab' after starting typing the filename and IPython will automatically fill it for you
# After this command a new variable will be created (spb_df) containing pandas dataframe with data in our file
# You can think of dataframe structure as an analogue of Excel sheet
spb_df = pd.read_table('spb.real.estate.archive.2018.tsv')

In [7]:
# HINT: if you don't know which arguments any function takes or what it does you can check the documenation right 
# in the notebook just by adding "?" after the function name. Let's look at pandas read_table function:
pd.read_table?

Signature:
pd.read_table(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols=None,
    squeeze: 'bool | None' = None,
    prefix: 'str | lib.NoDefault' = <no_default>,
    mangle_dupe_cols: 'bool' = True,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace: 'bool' = False,
    skiprows=None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values=None,
    keep_default_na: 'bool' = True,
    na_filter: 'bool' = True,
    verbose: 'bool' = False,
    skip_blank_lines: 'bool' = True,
    parse_dates=False,
    in

In [8]:
# let's check how much data in terms of rows we have where each row represent 
# one offer from Yandex.Realty
len(spb_df)

429187

In [9]:
# let's look at random sample of size 10 of the loaded dataset to understand what's inside
spb_df.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
91387,2044547480577462016,2015-08-12T00:00:00+03:00,2016-11-29T00:00:00+03:00,2800000.0,1,False,0,True,25.0,NaN,18.0,NaN,10.0,1,2,"Россия, Санкт-Петербург, улица Адмирала Черокова, 18к1",2641501526205844789
328779,271866666582052608,2017-07-04T00:00:00+03:00,2018-05-08T00:00:00+03:00,35000.0,6,False,2,False,50.0,8.0,31.0,50.0,7.0,2,2,"Россия, Санкт-Петербург, проспект Просвещения, 86к1",561327474932293436
74517,7695810247154147584,2016-07-07T00:00:00+03:00,2016-07-22T00:00:00+03:00,14000.0,7,False,1,False,38.0,10.0,19.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Красное Село, улица Спирина, 15",7268922991426436663
67500,244178448795566352,2016-04-04T00:00:00+03:00,2016-08-17T00:00:00+03:00,5790000.0,5,False,3,False,58.3,5.5,39.1,NaN,0.0,1,2,"Россия, Санкт-Петербург, Московский проспект, 220",4373184457909325875
285282,5544867011403410689,2018-02-03T00:00:00+03:00,2018-07-10T00:00:00+03:00,1450000.0,3,False,0,True,16.0,NaN,14.0,NaN,NaN,1,2,"Россия, Ленинградская область, Выборг, улица Сторожевой Башни, 13",104066620918480335
15545,4180194,2015-10-08T00:00:00+03:00,2016-02-01T00:00:00+03:00,4800000.0,3,False,2,False,44.0,6.2,26.7,NaN,3.0,1,2,"Россия, Санкт-Петербург, улица Ольги Форш, 13к1",861403378193386497
232981,2554639205139518221,2017-10-20T00:00:00+03:00,2018-02-07T00:00:00+03:00,2600000.0,15,False,1,False,40.0,9.7,18.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Старорусский проспект, 6",7423300724866729798
276202,8257798206279762944,2017-12-04T00:00:00+03:00,2018-02-08T00:00:00+03:00,1650000.0,1,False,0,True,14.5,NaN,NaN,NaN,3.0,1,2,"Россия, Санкт-Петербург, проспект Художников, 19к1",4652022804562841242
36074,9174606537143219602,2016-01-29T00:00:00+03:00,2016-10-09T00:00:00+03:00,4460000.0,19,False,1,False,33.8,8.9,17.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Адмирала Черокова, 18к3",2641501526205844791
88160,7110103192797359500,2016-09-06T00:00:00+03:00,2016-11-14T00:00:00+03:00,2550000.0,6,False,1,False,36.0,8.0,16.0,NaN,11.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Ленсоветовский, жилой комплекс Ленсоветовский",3809996798114979005


In [10]:
len(spb_df)

429187

In [11]:
spb_df.sample(5000).to_csv('spb.real.estate.archive.sample5000.tsv', index = False, sep='\t')

In [12]:
!pwd

/home/jovyan/e2e_ml


### Analyze prices for rent and sell
We see that we have both RENT and SELL data. 
<p>offer_type column contains data to distinct them, 2 stands for RENT, 1 for SELL


#### Analyze rent prices

In [13]:
# at first let's split our dataframe to rent and sell
# Select some part of dataframe according to conditions in brackets
rent_df = spb_df[spb_df.offer_type == 2]
sell_df = spb_df[spb_df.offer_type == 1]
# .format function allows you to put variales' values instead of {} in the final string
print("Total rent data size: {}".format(len(rent_df)))
print("Total sell data size: {}".format(len(sell_df)))

Total rent data size: 171186
Total sell data size: 258001


In [14]:
# let's check median and price for renting apartments
# .last_price indicates that you use data just from the column named last_price
rent_median_price = rent_df.last_price.median()
rent_mean_price = rent_df.last_price.mean()
print("Rent median price: {}".format(rent_median_price))
print("Rent mean price: {}".format(rent_mean_price))

Rent median price: 25000.0
Rent mean price: 31719.256638977487


In [15]:
# Just to look at data of one column you can select this column. 
# 'head' function will take only first rows 
rent_df.last_price.head(10)

1086    20000.0
1096    40000.0
1097    45000.0
1149    21000.0
1514    25000.0
1515    35000.0
1516    22000.0
1537    20000.0
1555    26000.0
1695    25000.0
Name: last_price, dtype: float64

#### Let's look at the most expensive apartments in rent

In [16]:
# .max function returns the maximum value across all rows
max_price_rent = rent_df.last_price.max()

In [17]:
print("Max price in rent: {}".format(max_price_rent))

Max price in rent: 4900000.0


In [18]:
# Now select all offers where price is equal to maximum value across all rent prices
rent_df_max = rent_df[rent_df.last_price == max_price_rent]

In [19]:
rent_df_max

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
123675,5594900220237251840,2017-01-16T00:00:00+03:00,2017-01-16T00:00:00+03:00,4900000.0,3,False,3,False,55.4,5.5,42.1,NaN,0.0,2,2,"Россия, Санкт-Петербург, Новочеркасский проспект, 36",7844665691371923607


We can see that it was probably a mistake. It's hard to imagine why an aparment could cost 4.9 mln rubles per month. To analyze it let's look at the random sample of rent offers with prices of more than 1 mln rubles.

In [20]:
# Knowing real estate market in St.Petersburg we understand 
# that offers with rent prices over 1 million rubles are outliers
outliers_rent = rent_df[rent_df.last_price > 1000000]
print("Number of outliers in rent: {}".format(len(outliers_rent)))

Number of outliers in rent: 22


In [21]:
outliers_rent.sample(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
83893,4075481786711135488,2016-08-18T00:00:00+03:00,2016-08-18T00:00:00+03:00,3500000.0,1,False,2,False,55.0,8.3,30.5,NaN,3.0,2,2,"Россия, Ленинградская область, Всеволожск, микрорайон Южный, Центральная улица, 13",4521154714631889065
123451,4266909851082118912,2017-01-14T00:00:00+03:00,2017-05-27T00:00:00+03:00,3100000.0,2,False,1,False,39.0,7.0,20.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, Петергоф, Озерковая улица, 39к1",6053955221850724766
289495,6929908963211630336,2018-02-10T00:00:00+03:00,2018-02-16T00:00:00+03:00,1800000.0,2,False,1,False,39.0,9.0,19.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, улица Цимбалина, 23",1935729367298454088
38416,5340729,2016-02-08T00:00:00+03:00,2016-02-15T00:00:00+03:00,2550000.0,8,False,1,False,38.0,6.0,19.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Варшавская улица, 51к1",6532707473577967135
409084,1523340115924662017,2018-07-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,1752250.0,21,False,2,False,38.0,13.0,25.0,50.0,10.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, улица Фёдора Абрамова, 23к1",4731794338099306507


In [22]:
# We see that all of these offers are results of user mistakes.
# Pause to think what kind of mistakes might cause such numbers?
# Let's now look at rent offers below 1M but more than 500K

In [23]:
rent_df[(rent_df.last_price > 500000) & (rent_df.last_price < 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
83621,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
89836,6302977522901450489,2016-09-14T00:00:00+03:00,2016-09-15T00:00:00+03:00,800000.0,2,False,2,False,80.0,20.0,40.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большая Морская улица, 27",8010764606439654926
106222,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
172166,3376106655979657514,2017-02-24T00:00:00+03:00,2017-05-24T00:00:00+03:00,600000.0,6,False,4,False,200.0,NaN,103.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к2",7861463534572863508
172950,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
253075,9174606537144832259,2017-11-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,600000.0,1,False,4,False,165.0,NaN,115.0,NaN,6.0,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355
321216,2596865999798823681,2018-03-31T00:00:00+03:00,2018-04-04T00:00:00+03:00,575000.0,3,False,4,False,120.0,NaN,NaN,50.0,1.0,2,2,"Россия, Санкт-Петербург, Кременчугская улица, 17к3",9078299753075959719
323711,9106659135022232833,2018-04-03T00:00:00+03:00,2018-06-15T00:00:00+03:00,600000.0,1,False,4,False,90.0,13.0,54.0,0.0,11.0,2,2,"Россия, Санкт-Петербург, Большой Сампсониевский проспект, 33/1",192308642065385452
336616,8867625833179319559,2018-04-21T00:00:00+03:00,2018-08-09T00:00:00+03:00,600000.0,1,False,4,False,117.0,20.0,83.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355
419166,5152264856292210972,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,600000.0,1,False,4,False,140.0,95.0,100.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355


In [24]:
# for these cases sometimes it seems that the price is correct - in the unified_address column we
# see that these offers are from elite houses
# like here: https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20проспект%20Динамо%2C%206/snyat/kvartira/
# but for some of them price seems to be 10 times more than it should be, like here
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Большая%20Морская%20улица%2C%2027/snyat/kvartira/
# let's find these cases

### Calculate price per square meter, get median prices for house and find outliers with the help of this

In [25]:
# Create a new column price_per_sq_m and calculate price per sq m
rent_df['price_per_sq_m'] = rent_df.last_price/rent_df.area

/tmp/ipykernel_468/816257644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rent_df['price_per_sq_m'] = rent_df.last_price/rent_df.area


In [26]:
rent_df.head(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m
1086,1100202,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,False,1,False,28.0,8.0,20.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,714.285714
1096,5028522699704970725,2015-01-25T00:00:00+03:00,2016-01-20T00:00:00+03:00,40000.0,4,False,2,False,72.0,10.0,44.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, набережная реки Фонтанки, 66",3406104650917898355,555.555556
1097,5028522699709767236,2015-01-25T00:00:00+03:00,2016-01-21T00:00:00+03:00,45000.0,2,False,2,False,55.0,NaN,33.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большой проспект П.С., 102",2357095685903407888,818.181818
1149,1100874,2015-01-28T00:00:00+03:00,2016-01-16T00:00:00+03:00,21000.0,5,False,1,False,35.0,8.0,18.0,NaN,11.0,2,2,"Россия, Санкт-Петербург, Лужская улица, 8",5887402119500981764,600.000000
1514,875413,2015-02-13T00:00:00+03:00,2016-01-23T00:00:00+03:00,25000.0,10,False,1,True,44.0,15.0,18.0,90.0,10.0,2,2,"Россия, Санкт-Петербург, Учительская улица, 3",6527910144166454997,568.181818


In [27]:
# find what's median and mean price per square meter in rent
median_price_per_sq_m = rent_df.price_per_sq_m.median()
mean_price_per_sq_m = rent_df.price_per_sq_m.mean()
print("Median price per sq m in rent: {}".format(median_price_per_sq_m))
print("Mean price per sq m in rent: {}".format(mean_price_per_sq_m))

Median price per sq m in rent: 550.0
Mean price per sq m in rent: 600.1110692967684


#### Find median price per sq m per house

In [28]:
# .groupby function groups dataframe by a selected column (unified_address)
# For each address we then take a median of price per sq m and return a new dataframe 
# with a result
house_rent_df = rent_df.groupby('unified_address').price_per_sq_m.median().reset_index()

In [29]:
house_rent_df.head(5)

,unified_address,price_per_sq_m
0,"Россия, Ленинградская область, Бокситогорск, Комсомольская улица, 5А",171.428571
1,"Россия, Ленинградская область, Бокситогорск, Южная улица, 15",250.000000
2,"Россия, Ленинградская область, Бокситогорск, улица Металлургов, 7",164.948454
3,"Россия, Ленинградская область, Бокситогорск, улица Павлова, 37",138.888889
4,"Россия, Ленинградская область, Бокситогорский район, Пикалёво, 6-й микрорайон, 25",195.652174


In [30]:
# Rename the column with a 'rename' function in this new dataframe
house_rent_df.rename(columns = {'price_per_sq_m': 'house_price_sqm_median'}, inplace = True)

#### Merge rent data with house median prices and inspect outliers

In [31]:
# With a 'merge' function join two dataframes.
# As these 2 dataframes have only one common column name (unified_address) it will merge dataframes by it
rent_df = rent_df.merge(house_rent_df)

In [32]:
# Assign outliers if price per sq m is 5 times higher than the median value in the house
outliers = rent_df[(rent_df.price_per_sq_m/rent_df.house_price_sqm_median) > 5]

In [33]:
print(len(outliers))

49


In [34]:
outliers.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
126900,2877221490003907329,2018-01-10T00:00:00+03:00,2018-02-09T00:00:00+03:00,200000.0,5,False,1,False,40.0,NaN,NaN,NaN,NaN,2,2,"Россия, Санкт-Петербург, улица Есенина, 14к2",7215469126417656128,5000.000000,606.060606
80391,5565310413830597669,2018-06-19T00:00:00+03:00,2018-06-26T00:00:00+03:00,200000.0,13,False,0,True,40.0,NaN,21.5,100.0,NaN,2,2,"Россия, Ленинградская область, Всеволожский район, деревня Новое Девяткино, Арсенальная улица, 7",1966219418622870596,5000.000000,452.272727
42169,5516818252822041857,2018-02-17T00:00:00+03:00,2018-03-10T00:00:00+03:00,350000.0,4,False,3,False,100.0,15.0,65.0,0.0,3.0,2,2,"Россия, Санкт-Петербург, Таврическая улица, 2",266291655388673730,3500.000000,694.291755
73297,5365373,2016-02-10T00:00:00+03:00,2016-02-17T00:00:00+03:00,180000.0,6,False,1,False,35.0,9.0,20.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, улица Крыленко, 19к1",4951256661585230565,5142.857143,562.500000
50140,6302977522901450489,2016-09-14T00:00:00+03:00,2016-09-15T00:00:00+03:00,800000.0,2,False,2,False,80.0,20.0,40.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большая Морская улица, 27",8010764606439654926,10000.000000,841.121495
74207,6589696843731071745,2018-03-12T00:00:00+03:00,2018-04-02T00:00:00+03:00,2500000.0,2,False,4,False,150.0,53.0,47.0,50.0,NaN,2,2,"Россия, Санкт-Петербург, Волховский переулок, 4",4642914072625498389,16666.666667,887.850467
24222,3988133,2015-09-19T00:00:00+03:00,2016-04-13T00:00:00+03:00,3450000.0,19,False,1,True,30.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, Заречная улица, 19к1",2079266574874175179,115000.000000,571.428571
109040,7710633944129267457,2017-08-07T00:00:00+03:00,2017-11-07T00:00:00+03:00,4500000.0,7,False,1,False,63.1,16.1,23.4,NaN,0.0,2,2,"Россия, Санкт-Петербург, Колпино, улица Ижорского Батальона, 18",6064043689074707027,71315.372425,397.674419
793,5340729,2016-02-08T00:00:00+03:00,2016-02-15T00:00:00+03:00,2550000.0,8,False,1,False,38.0,6.0,19.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Варшавская улица, 51к1",6532707473577967135,67105.263158,555.555556
108994,7521881594612635746,2018-05-14T00:00:00+03:00,2018-05-21T00:00:00+03:00,4700000.0,10,False,1,False,40.3,8.3,20.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, Ленинский проспект, 111к2",2371480959169104773,116625.310174,602.631579


#### Clean data from the outliers

In [35]:
# Let's create a new dataframe with clean data.
# With ~ we can select the data which does NOT correspond to the required conditions
rent_df_cleaned = rent_df[~((rent_df.price_per_sq_m/rent_df.house_price_sqm_median) > 5)]

In [36]:
# check that in the middle segment everything is fine now
rent_df_cleaned[(rent_df_cleaned.last_price > 500000) & (rent_df_cleaned.last_price < 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
95391,3376106655979657514,2017-02-24T00:00:00+03:00,2017-05-24T00:00:00+03:00,600000.0,6,False,4,False,200.0,NaN,103.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к2",7861463534572863508,3000.000000,1385.501355
96070,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3055.555556,1442.307692
96075,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3208.556150,1442.307692
96091,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692
154614,9174606537144832259,2017-11-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,600000.0,1,False,4,False,165.0,NaN,115.0,NaN,6.0,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,3636.363636,3636.363636
154615,8867625833179319559,2018-04-21T00:00:00+03:00,2018-08-09T00:00:00+03:00,600000.0,1,False,4,False,117.0,20.0,83.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,5128.205128,3636.363636
154617,5152264856292210972,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,600000.0,1,False,4,False,140.0,95.0,100.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,4285.714286,3636.363636


In [37]:
# look at offers with price > 1 000 000
rent_df_cleaned[(rent_df_cleaned.last_price >= 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
67005,5199154,2016-01-25T00:00:00+03:00,2016-01-26T00:00:00+03:00,3800000.0,3,False,2,False,51.0,6.0,30.5,NaN,0.0,2,2,"Россия, Санкт-Петербург, Пискарёвский проспект, 149",7757194067653498771,74509.803922,74509.803922
95935,8743101202301731585,2016-10-13T00:00:00+03:00,2016-11-12T00:00:00+03:00,1500000.0,3,False,2,False,54.7,NaN,NaN,NaN,7.0,2,2,"Россия, Ленинградская область, Выборгский район, Светогорское городское поселение, поселок городского типа Лесогорский, улица Труда, 7",2755098623625145659,27422.303473,13811.700183
114024,4075481786711135488,2016-08-18T00:00:00+03:00,2016-08-18T00:00:00+03:00,3500000.0,1,False,2,False,55.0,8.3,30.5,NaN,3.0,2,2,"Россия, Ленинградская область, Всеволожск, микрорайон Южный, Центральная улица, 13",4521154714631889065,63636.363636,63636.363636
164662,4088017598264857345,2018-02-09T00:00:00+03:00,2018-03-21T00:00:00+03:00,1330000.0,3,False,1,False,33.0,6.0,19.0,NaN,3.0,2,2,"Россия, Ленинградская область, Луга, проспект Урицкого, 65",1792010144615909485,40303.030303,40303.030303


In [38]:
# we see, that all of them are outliers, they were not cleaned by the house price per sq m median 
# because it was the only one or just one of two offers per house
# let's clean them also
# Store result in the same dataframe
rent_df_cleaned = rent_df_cleaned[rent_df_cleaned.last_price < 1000000]

In [39]:
# let's examine apartments with big price per sq m
rent_df_cleaned[rent_df_cleaned.price_per_sq_m > 3000]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
549,8596035942924139265,2018-05-17T00:00:00+03:00,2018-07-04T00:00:00+03:00,450000.0,6,False,2,False,106.0,15.0,65.0,30.0,NaN,2,2,"Россия, Санкт-Петербург, Воскресенская набережная, 4",7262659557693263963,4245.283019,975.609756
28182,1492050389917501301,2017-06-07T00:00:00+03:00,2017-06-07T00:00:00+03:00,195000.0,4,False,1,False,57.0,8.0,19.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, проспект Добролюбова, 2",6684242979485775735,3421.052632,779.220779
39213,3793550987719225600,2017-03-19T00:00:00+03:00,2017-11-02T00:00:00+03:00,310000.0,3,False,4,False,100.0,14.0,71.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Лиговский проспект, 123",2861706125080798064,3100.000000,976.307190
44574,1896730477284436579,2017-05-31T00:00:00+03:00,2017-05-31T00:00:00+03:00,230000.0,9,False,2,False,67.0,14.0,30.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Глухая Зеленина улица, 6",6445545084936583797,3432.835821,923.076923
96070,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3055.555556,1442.307692
96075,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3208.556150,1442.307692
96091,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692
99409,7102677476104578816,2016-05-20T00:00:00+03:00,2016-05-20T00:00:00+03:00,270000.0,1,False,1,False,80.0,NaN,NaN,0.0,3.0,2,2,"Россия, Санкт-Петербург, Загородный проспект, 5",8810747501928407960,3375.000000,1978.197674
104668,159653575951184384,2016-06-16T00:00:00+03:00,2016-07-01T00:00:00+03:00,300000.0,2,False,3,False,54.0,8.0,35.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, 18-я линия В.О., 43",6003296125598219505,5555.555556,5555.555556
150701,1492050390917641037,2016-10-28T00:00:00+03:00,2017-06-07T00:00:00+03:00,400000.0,4,False,3,False,130.0,13.0,85.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Константиновский проспект, 23",5091810811209082905,3076.923077,3484.848485


In [40]:
# we can see that outliers are those which are in houses with price per sq m below 1000 rubles, 
# and those which are the same with median, in which case it's the only offer per house
outliers_count = len(rent_df_cleaned[(rent_df_cleaned.price_per_sq_m > 3000) 
                                     & ((rent_df_cleaned.house_price_sqm_median < 1000) 
                                        | (rent_df_cleaned.house_price_sqm_median == rent_df_cleaned.price_per_sq_m))])
print("outliers found: {}".format(outliers_count))
rent_df_cleaned = rent_df_cleaned[~((rent_df_cleaned.price_per_sq_m > 3000) 
                                     & ((rent_df_cleaned.house_price_sqm_median < 1000) 
                                        | (rent_df_cleaned.house_price_sqm_median == rent_df_cleaned.price_per_sq_m)))]

outliers found: 8


In [41]:
# now max price should be correct, lets look at this offer
max_rent_apartment = rent_df_cleaned[rent_df_cleaned.last_price == rent_df_cleaned.last_price.max()]
max_rent_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
96091,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692


In [42]:
# let's look at house address. 
# with 'iloc' property we can take any row by index, 
# in this case we take the first item to get the address
max_rent_apartment.unified_address.iloc[0]

'Россия, Санкт-Петербург, Парадная улица, 3к1'

In [43]:
# impoort urllib to prepare correct URLs for looking at offers archive 
# at https://realty.yandex.ru/archive
import urllib

In [44]:
# if you're interested in images of this apartment you can find it here: 
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Парадная%20улица%2C%203к1/snyat/kvartira/2/?roomsTotal=4

# let's print the URL of the page, on which we can find our offer
# we use urllib.parse.quote function to get URL part from our address, otherwise URL wouldn't be correct
# print("https://realty.yandex.ru/archive/{}/snyat/kvartira/2/?roomsTotal=4".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
# print("https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/{}/snyat/kvartira/?roomsTotal=4".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
print("https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/{}/snyat/kvartira/".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
# use filters there to select only apartments with 4 rooms

https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%9F%D0%B0%D1%80%D0%B0%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%2C%203%D0%BA1/snyat/kvartira/


### Find houses with the most cheapest and the most expensive rent price

In [45]:
# Create a new dataframe with houses and median price per sq m
# This time it should with the data calculated without outliers
house_rent_df_cleaned = rent_df_cleaned.groupby('unified_address').price_per_sq_m.median().reset_index()
house_rent_df_cleaned.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)

In [46]:
most_expensive_house = house_rent_df_cleaned[house_rent_df_cleaned.house_price_sqm_median_cleaned == house_rent_df_cleaned.house_price_sqm_median_cleaned.max()]
most_expensive_house

,unified_address,house_price_sqm_median_cleaned
16851,"Россия, Санкт-Петербург, проспект Динамо, 6",3705.357143


In [47]:
# you can look at the most expensive house here
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20проспект%20Динамо%2C%206/snyat/kvartira/
print("https://realty.yandex.ru/archive/{}/snyat/kvartira/".format(urllib.parse.quote(most_expensive_house.unified_address.iloc[0])))

https://realty.yandex.ru/archive/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82%20%D0%94%D0%B8%D0%BD%D0%B0%D0%BC%D0%BE%2C%206/snyat/kvartira/


#### Find the most cheapest house for rent

In [48]:
most_cheapest_house = house_rent_df_cleaned[house_rent_df_cleaned.house_price_sqm_median_cleaned == house_rent_df_cleaned.house_price_sqm_median_cleaned.min()]
most_cheapest_house

,unified_address,house_price_sqm_median_cleaned
2421,"Россия, Ленинградская область, Кириши, улица Мира, 14",50.0


In [49]:
# we see that it's located in Leningrad Oblast, let's look only in St. Petersburg data
house_rent_df_cleaned_spb = house_rent_df_cleaned[house_rent_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
house_rent_df_cleaned_spb[house_rent_df_cleaned_spb.house_price_sqm_median_cleaned == house_rent_df_cleaned_spb.house_price_sqm_median_cleaned.min()]

,unified_address,house_price_sqm_median_cleaned
4852,"Россия, Санкт-Петербург, Большая Зеленина улица, 12",76.923077


#### Clean cheap outliers
We see that we should clean the data, the cheapest house above is definitely a mistake, and it is an underground station building. You can check it at https://maps.yandex.ru

In [50]:
# consider only data inside the city without oblast
rent_df_cleaned_spb = rent_df_cleaned[rent_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
median_price_per_sq_m_in_spb = rent_df_cleaned_spb.price_per_sq_m.median()
print("Median price per sq m in St. Petersburg in rent: {}".format(median_price_per_sq_m_in_spb))

Median price per sq m in St. Petersburg in rent: 555.5555555555555


In [51]:
# look at cheap apartments which are more than two times cheaper 
# than the median value of offers we calculated above (555)
rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m < 250].sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
93985,3073129119751288064,2016-04-22T00:00:00+03:00,2016-09-13T00:00:00+03:00,18000.0,7,False,3,False,84.0,12.0,56.0,100.0,11.0,2,2,"Россия, Санкт-Петербург, Окраинная улица, 9Д",4510539508387660404,214.285714,225.000000
120838,5505393979993633025,2017-04-20T00:00:00+03:00,2017-05-08T00:00:00+03:00,21500.0,8,False,3,False,90.0,17.0,75.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,238.888889,264.912281
65548,5597875438972269568,2017-08-09T00:00:00+03:00,2017-12-04T00:00:00+03:00,22000.0,8,False,4,False,92.0,NaN,64.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Маршака, 4",4203038512602532676,239.130435,288.002048
50180,2629908937451490817,2017-02-20T00:00:00+03:00,2017-02-26T00:00:00+03:00,15000.0,3,False,2,False,63.0,12.0,38.0,80.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Колпинское шоссе, 34к1",4081548880280501571,238.095238,258.064516
108545,7032793694426910721,2016-11-28T00:00:00+03:00,2016-12-04T00:00:00+03:00,15000.0,6,False,1,False,62.0,12.0,35.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 19/3",6689430674378351199,241.935484,245.901639
96184,5836091506479056129,2016-05-14T00:00:00+03:00,2016-05-24T00:00:00+03:00,16000.0,7,False,3,False,80.0,12.0,48.0,80.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Полоцкая улица, 3",5139440287095964726,200.000000,225.000000
26991,9206218891413373697,2017-03-07T00:00:00+03:00,2017-03-07T00:00:00+03:00,130.0,3,False,3,False,108.0,47.0,NaN,50.0,11.0,2,2,"Россия, Санкт-Петербург, набережная канала Грибоедова, 12",3976567739161800354,1.203704,982.558140
120897,6840376369271565824,2018-04-25T00:00:00+03:00,2018-08-16T00:00:00+03:00,18000.0,5,False,3,False,86.0,15.0,NaN,50.0,11.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,209.302326,264.912281
120846,1570606376708378368,2017-11-29T00:00:00+03:00,2018-02-06T00:00:00+03:00,19000.0,8,False,3,False,82.0,14.0,48.0,0.0,11.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,231.707317,264.912281
154295,8338419857397135205,2017-08-01T00:00:00+03:00,2017-08-18T00:00:00+03:00,16000.0,6,False,2,False,65.0,13.0,34.0,60.0,11.0,2,2,"Россия, Санкт-Петербург, территория Горелово, Красносельское шоссе, 52к1",1035767884199115631,246.153846,246.153846


In [52]:
# it seems that we should clean all apartments where median price is at least 2x higher and cheap 
outliers = rent_df_cleaned_spb[(rent_df_cleaned_spb.price_per_sq_m < 250) 
                               & (rent_df_cleaned_spb.house_price_sqm_median/rent_df_cleaned_spb.price_per_sq_m >= 2)]

In [53]:
print("number of cheap outliers 2x cheaper: {}".format(len(outliers)))

number of cheap outliers 2x cheaper: 100


In [54]:
# clean outliers
rent_df_cleaned_spb = rent_df_cleaned_spb[~((rent_df_cleaned_spb.price_per_sq_m < 250) 
                               & (rent_df_cleaned_spb.house_price_sqm_median/rent_df_cleaned_spb.price_per_sq_m >= 2))]

In [55]:
# look at cheap apartments which were not cleaned with the previous heuristics
rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m < 200]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
46867,4857633,2015-12-13T00:00:00+03:00,2016-06-05T00:00:00+03:00,12000.0,6,False,3,False,61.0,7.0,42.0,80.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Стрельнинское шоссе, 6к3",1523555400005784785,196.721311,256.578947
57861,4975278,2015-12-27T00:00:00+03:00,2016-01-12T00:00:00+03:00,13000.0,4,False,3,False,78.0,10.0,47.0,NaN,10.0,2,2,"Россия, Санкт-Петербург, 12-я Красноармейская улица, 7",6204918944542168630,166.666667,166.666667
80404,2939523434608526081,2018-03-06T00:00:00+03:00,2018-03-13T00:00:00+03:00,15000.0,6,False,3,False,87.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, Окраинная улица, 9В",4510539508387660402,172.413793,241.715116
85844,249021112485953840,2016-03-21T00:00:00+03:00,2016-07-05T00:00:00+03:00,12000.0,3,False,4,False,112.0,14.0,16.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Свеаборгская улица, 23",4943588578198944211,107.142857,107.142857
91302,8176284694677069056,2016-05-18T00:00:00+03:00,2016-05-18T00:00:00+03:00,14000.0,4,False,3,False,74.0,10.0,NaN,50.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 6к3",4976238169808254780,189.189189,291.963016
101249,5379909360861020417,2016-06-02T00:00:00+03:00,2016-06-09T00:00:00+03:00,14000.0,3,False,3,False,80.0,12.0,NaN,80.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 27",5194982532050895016,175.000000,233.333333
105186,801981697508642560,2016-06-20T00:00:00+03:00,2016-06-29T00:00:00+03:00,12000.0,6,False,5,False,156.0,23.0,16.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Большая Зеленина улица, 12",7137635118733633172,76.923077,76.923077
105957,6302976505020612958,2016-06-25T00:00:00+03:00,2016-11-08T00:00:00+03:00,17000.0,4,False,3,False,100.0,16.0,20.0,50.0,0.0,2,2,"Россия, Санкт-Петербург, Центральный район, улица Некрасова, 38",4677861887331789522,170.000000,170.000000
106615,3653332947260070401,2016-07-01T00:00:00+03:00,2016-07-08T00:00:00+03:00,11000.0,3,False,2,False,71.0,9.0,42.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",4011485438398100794,154.929577,183.098592
120839,2164881834443110913,2017-05-07T00:00:00+03:00,2017-05-14T00:00:00+03:00,15000.0,5,False,3,False,93.0,10.0,54.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,161.290323,264.912281


In [56]:
# there are cases where houses had just one offer and house median equals offer's price per square meter
# let's remove these outliers
rent_df_cleaned_spb = rent_df_cleaned_spb[~((rent_df_cleaned_spb.price_per_sq_m < 200) 
                                          & (rent_df_cleaned_spb.price_per_sq_m == rent_df_cleaned_spb.house_price_sqm_median))]

In [57]:
# look at the cheapest apartment
cheapest_offer = rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m == rent_df_cleaned_spb.price_per_sq_m.min()]
cheapest_offer

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
106615,3653332947260070401,2016-07-01T00:00:00+03:00,2016-07-08T00:00:00+03:00,11000.0,3,False,2,False,71.0,9.0,42.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",4011485438398100794,154.929577,183.098592


#### Recalculate houses price medians on the cleaned data and find the cheapest house

In [58]:
house_rent_df_cleaned_spb = rent_df_cleaned_spb.groupby('unified_address').price_per_sq_m.median().reset_index()
house_rent_df_cleaned_spb.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)
cheapest_house = house_rent_df_cleaned_spb[house_rent_df_cleaned_spb.house_price_sqm_median_cleaned == house_rent_df_cleaned_spb.house_price_sqm_median_cleaned.min()]
cheapest_house

,unified_address,house_price_sqm_median_cleaned
5245,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",183.098592


In [59]:
# looking at the map we understand why it's so cheap: it's far away from the center of the city
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Красное%20Село%2C%20Нагорная%20улица%2C%2037/snyat/kvartira/
print("https://realty.yandex.ru/archive/{}/snyat/kvartira/".format(urllib.parse.quote(cheapest_house.unified_address.iloc[0])))

https://realty.yandex.ru/archive/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B5%20%D0%A1%D0%B5%D0%BB%D0%BE%2C%20%D0%9D%D0%B0%D0%B3%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%2C%2037/snyat/kvartira/


### Analyze how many offers have the commission 

In [60]:
# Some offers has commision fee for the agent (agent_fee)
# With 'value_counts' function find out the share of corresponding value
rent_df_cleaned_spb.agent_fee.value_counts(normalize=True, dropna=False)

100.000000    0.328129
50.000000     0.293380
NaN           0.209340
0.000000      0.035378
90.000000     0.034537
                ...   
14.000000     0.000006
11.000000     0.000006
17.000000     0.000006
92.000000     0.000006
64.997665     0.000006
Name: agent_fee, Length: 102, dtype: float64

We see that the most popular commission is 100% - 33% of offers. 21% of offers don't have any information about commission.

### Self-control stops
1. Calculate median and mean prices for apartments for rent after cleaning the data in St.Petersburg without Leningrad Oblast. Which of the statistics changed more and why?
2. Calculate median and mean prices for apartments for sell before cleaning the data.
3. Find houses with the most cheapest and most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers.
4. Find the most expensive and the most cheapest apartment in St. Petersburg after cleaning outliers.
5. Calculate how many years does it take to cover all money spent on buying apartment by renting it. Find houses in which it's the most effective to invest in buying if you plan to rent and houses in which it will be the most ineffective.

In [61]:
rent_df_cleaned_spb.head()

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
0,1100202,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,False,1,False,28.0,8.0,20.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,714.285714,512.820513
1,4593125,2015-11-17T00:00:00+03:00,2016-03-04T00:00:00+03:00,24000.0,4,False,2,False,59.0,10.0,35.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,406.779661,512.820513
2,4586184,2015-11-17T00:00:00+03:00,2016-04-24T00:00:00+03:00,18000.0,6,False,1,False,36.0,9.0,17.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,500.000000,512.820513
3,5308071,2016-02-04T00:00:00+03:00,2016-02-28T00:00:00+03:00,18000.0,8,False,1,False,39.0,10.0,18.0,90.0,0.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,461.538462,512.820513
4,7534283612538021121,2016-02-28T00:00:00+03:00,2016-04-02T00:00:00+03:00,19000.0,3,False,1,False,36.0,8.0,18.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,527.777778,512.820513


1. Median and mean prices for apartments for rent after cleaning the data in St.Petersburg without Leningrad Oblast

In [62]:
median_price_per_sq_m = rent_df_cleaned_spb.price_per_sq_m.median()
mean_price_per_sq_m = rent_df_cleaned_spb.price_per_sq_m.mean()
print("Median price per sq m in rent: {}".format(median_price_per_sq_m))
print("Mean price per sq m in rent: {}".format(mean_price_per_sq_m))

Median price per sq m in rent: 555.5555555555555
Mean price per sq m in rent: 600.9353049799272


In [63]:
median_price = rent_df_cleaned_spb.last_price.median()
mean_price = rent_df_cleaned_spb.last_price.mean()
print("Median price in rent: {}".format(median_price))
print("Mean price in rent: {}".format(mean_price))

Median price in rent: 25000.0
Mean price in rent: 32605.087189685033


before cleaning:
Rent median price: 25000.0
Rent mean price: 31719.256638977487

The mean price became lower because we removed the outliers which obviously influenced the mean price much more than they influenced the median.

2. Calculate median and mean prices for apartments for sell before cleaning the data.

In [64]:
sell_median_price = sell_df.last_price.median()
sell_mean_price = sell_df.last_price.mean()
print("Sell median price: {}".format(sell_median_price))
print("Sell mean price: {}".format(sell_mean_price))

Sell median price: 4599000.0
Sell mean price: 6969288.146034318


Outliers cleaning

In [65]:
max_price_sell = sell_df.last_price.max()
min_price_sell = sell_df.last_price.min()

print("Max price in sell: {}".format(max_price_sell))
print("Min price in sell: {}".format(min_price_sell))

Max price in sell: 763000000.0
Min price in sell: 19000.0


In [66]:
sell_df_max = sell_df[sell_df.last_price == max_price_sell]

In [67]:
sell_df_max

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
224434,619165944711730944,2017-09-30T00:00:00+03:00,2017-11-02T00:00:00+03:00,763000000.0,10,False,7,False,400.0,NaN,250.0,NaN,1.0,1,2,"Россия, Санкт-Петербург, Мичуринская улица, 4",7962972499404580673


In [68]:
sell_df_min = sell_df[sell_df.last_price == min_price_sell]

In [69]:
sell_df_min

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
70001,3438368921946229505,2016-06-16T00:00:00+03:00,2016-06-19T00:00:00+03:00,19000.0,4,False,1,True,39.0,NaN,18.0,NaN,11.0,1,2,"Россия, Санкт-Петербург, улица Ленсовета, 69к1",8047169421659571687


In [70]:
sell_df['price_per_sq_m'] = sell_df.last_price/sell_df.area

/tmp/ipykernel_468/2962634315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_df['price_per_sq_m'] = sell_df.last_price/sell_df.area


In [71]:
sell_df.head(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m
0,5490504238877583424,2014-11-27T00:00:00+03:00,2016-03-14T00:00:00+03:00,3676740.0,19,False,1,False,27.40,16.56,16.56,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3552413344225333068,134187.591241
1,5490504238903444021,2014-11-27T00:00:00+03:00,2016-04-28T00:00:00+03:00,7038880.0,2,False,3,False,85.84,19.64,41.20,NaN,0.0,1,2,"Россия, Санкт-Петербург, Кушелевская дорога, 3к4",7274736308217971906,82000.000000
2,5490504238903447743,2014-11-27T00:00:00+03:00,2016-05-11T00:00:00+03:00,7038880.0,3,False,3,False,85.84,19.64,41.20,NaN,0.0,1,2,"Россия, Санкт-Петербург, Кушелевская дорога, 3к4",7274736308217971906,82000.000000
3,5490504238877673946,2014-11-27T00:00:00+03:00,2016-05-12T00:00:00+03:00,12074400.0,5,False,3,False,93.60,10.90,52.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, Свердловская набережная, 58А",5917543958366039686,129000.000000
4,5490504238877648740,2014-11-27T00:00:00+03:00,2016-05-12T00:00:00+03:00,20406100.0,7,False,4,False,142.70,13.84,82.55,NaN,0.0,1,2,"Россия, Санкт-Петербург, Свердловская набережная, 58А",5917543958366039686,143000.000000


In [72]:
median_price_per_sq_m = sell_df.price_per_sq_m.median()
mean_price_per_sq_m = sell_df.price_per_sq_m.mean()
print("Median price per sq m in sell: {}".format(median_price_per_sq_m))
print("Mean price per sq m in sell: {}".format(mean_price_per_sq_m))

Median price per sq m in sell: 95214.28571428571
Mean price per sq m in sell: 102713.88007995683


In [73]:
# For each address we then take a median of price per sq m and return a new dataframe with a result
house_sell_df = sell_df.groupby('unified_address').price_per_sq_m.median().reset_index()

In [74]:
house_sell_df.head(5)

,unified_address,price_per_sq_m
0,"Россия, Ленинградская область, Бокситогорск, Городская улица, 1",15316.249610
1,"Россия, Ленинградская область, Бокситогорск, Городская улица, 3",19080.327869
2,"Россия, Ленинградская область, Бокситогорск, Городская улица, 4",20812.534664
3,"Россия, Ленинградская область, Бокситогорск, Дымское шоссе, 1",14600.000000
4,"Россия, Ленинградская область, Бокситогорск, Заводская улица, 11",14642.978290


In [75]:
house_sell_df.rename(columns = {'price_per_sq_m': 'house_price_sqm_median'}, inplace = True)

In [76]:
sell_df = sell_df.merge(house_sell_df)

In [77]:
# Assign outliers if price per sq m is 5 times higher than the median value in the house
outliers = sell_df[(sell_df.price_per_sq_m/sell_df.house_price_sqm_median) > 5]

In [78]:
print(len(outliers))

52


In [79]:
outliers.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
256110,5485935064186424652,2018-06-27T00:00:00+03:00,2018-08-20T00:00:00+03:00,24412900.0,4,False,1,False,38.12,10.70,16.83,NaN,5.0,1,2,"Россия, Ленинградская область, Всеволожск, микрорайон Южный, улица Доктора Сотникова, 31",2624586177104353974,6.404224e+05,60355.907069
201018,6543132636567265536,2017-08-07T00:00:00+03:00,2017-09-21T00:00:00+03:00,285000000.0,1,False,1,False,30.70,6.50,17.50,NaN,7.0,1,2,"Россия, Санкт-Петербург, проспект Космонавтов, 48к3",9137473764455367897,9.283388e+06,100274.725275
124501,216145565898896384,2016-12-20T00:00:00+03:00,2017-01-19T00:00:00+03:00,59401500.0,5,False,3,False,55.00,6.00,39.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Ленинский проспект, 172",5890508792580294584,1.080027e+06,89263.392857
219633,6633140150908361473,2018-02-20T00:00:00+03:00,2018-04-19T00:00:00+03:00,68000000.0,5,False,3,False,83.00,9.00,60.00,NaN,NaN,1,2,"Россия, Санкт-Петербург, Средний проспект Васильевского острова, 49",5553660887271054669,8.192771e+05,128160.919540
183908,5614702324467077377,2016-12-08T00:00:00+03:00,2017-01-24T00:00:00+03:00,11500000.0,4,False,2,False,54.00,9.00,30.00,NaN,8.0,1,2,"Россия, Ленинградская область, Сланцевское городское поселение, Сланцы, улица Ленина, 24А",1341663197783124531,2.129630e+05,18271.877826
170482,3367195771329321292,2016-06-09T00:00:00+03:00,2016-09-13T00:00:00+03:00,86000000.0,3,False,4,False,75.20,8.30,53.60,NaN,0.0,1,2,"Россия, Санкт-Петербург, Думская улица, 5/22",7300988142191185101,1.143617e+06,148706.896552
144136,201426560789662465,2016-09-27T00:00:00+03:00,2016-10-03T00:00:00+03:00,40000000.0,3,False,2,False,45.89,7.03,28.50,NaN,2.0,1,2,"Россия, Санкт-Петербург, проспект Энгельса, 151к1",6711366030409608752,8.716496e+05,91506.602641
162440,5153034022900498092,2017-11-15T00:00:00+03:00,2018-01-20T00:00:00+03:00,57000000.0,4,False,3,False,63.00,7.20,41.30,NaN,NaN,1,2,"Россия, Санкт-Петербург, проспект Ветеранов, 142",6845097171321796802,9.047619e+05,91904.761905
147359,5902825275847175680,2016-03-24T00:00:00+03:00,2016-06-22T00:00:00+03:00,34000000.0,10,False,1,False,40.40,NaN,14.00,NaN,11.0,1,2,"Россия, Санкт-Петербург, переулок Гривцова, 6",6312852829589816564,8.415842e+05,99509.803922
120479,5152132572093402629,2016-12-23T00:00:00+03:00,2016-12-26T00:00:00+03:00,92894184.0,3,False,1,False,45.59,11.86,20.47,NaN,0.0,1,2,"Россия, Ленинградская область, Всеволожский район, Колтушское сельское поселение, деревня Старая, Школьный переулок, 3",3653851415933114236,2.037600e+06,55100.000000


In [80]:
# With ~ we can select the data which does NOT correspond to the required conditions
sell_df_cleaned = sell_df[~((sell_df.price_per_sq_m/sell_df.house_price_sqm_median) > 5)]

In [81]:
sell_df_cleaned = sell_df_cleaned[sell_df_cleaned.last_price > 1000000]

In [82]:
outliers_count = len(sell_df_cleaned[(sell_df_cleaned.price_per_sq_m > 500000) 
                                     & ((sell_df_cleaned.house_price_sqm_median < 200000) 
                                        | (sell_df_cleaned.house_price_sqm_median == sell_df_cleaned.price_per_sq_m))])
print("outliers found: {}".format(outliers_count))
sell_df_cleaned = sell_df_cleaned[~((sell_df_cleaned.price_per_sq_m > 500000) 
                                     & ((sell_df_cleaned.house_price_sqm_median < 200000) 
                                        | (sell_df_cleaned.house_price_sqm_median == sell_df_cleaned.price_per_sq_m)))]

outliers found: 49


In [83]:
max_sell_apartment = sell_df_cleaned[sell_df_cleaned.last_price == sell_df_cleaned.last_price.max()]
max_sell_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
237351,8310251211233707984,2018-03-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,605740032.0,2,False,5,False,443.0,10.0,248.0,0.0,NaN,1,2,"Россия, Санкт-Петербург, Южная дорога, 5",3312559599398519904,1.367359e+06,759301.442673


In [84]:
max_sell_apartment.unified_address.iloc[0]

'Россия, Санкт-Петербург, Южная дорога, 5'

In [85]:
house_sell_df_cleaned = sell_df_cleaned.groupby('unified_address').price_per_sq_m.median().reset_index()
house_sell_df_cleaned.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)

In [86]:
most_expensive_house = house_sell_df_cleaned[house_sell_df_cleaned.house_price_sqm_median_cleaned == house_sell_df_cleaned.house_price_sqm_median_cleaned.max()]
most_expensive_house

,unified_address,house_price_sqm_median_cleaned
22292,"Россия, Санкт-Петербург, Санаторная аллея, 3",818152.92298


In [87]:
most_cheapest_house = house_sell_df_cleaned[house_sell_df_cleaned.house_price_sqm_median_cleaned == house_sell_df_cleaned.house_price_sqm_median_cleaned.min()]
most_cheapest_house

,unified_address,house_price_sqm_median_cleaned
15518,"Россия, Санкт-Петербург, Конторская улица, 16",5000.0


In [88]:
#let's clean Len Oblast from our data
house_sell_df_cleaned_spb = house_sell_df_cleaned[house_sell_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
house_sell_df_cleaned_spb[house_sell_df_cleaned_spb.house_price_sqm_median_cleaned == house_sell_df_cleaned_spb.house_price_sqm_median_cleaned.min()]

,unified_address,house_price_sqm_median_cleaned
15518,"Россия, Санкт-Петербург, Конторская улица, 16",5000.0


In [89]:
sell_df_cleaned_spb = sell_df_cleaned[sell_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
median_price_per_sq_m_in_spb = sell_df_cleaned_spb.price_per_sq_m.median()
print("Median price per sq m in St. Petersburg in sell: {}".format(median_price_per_sq_m_in_spb))

Median price per sq m in St. Petersburg in sell: 102272.72727272728


In [90]:
# look at cheap apartments which are more than two times cheaper than the median value of offers we calculated above (102272)
sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m < 50000].sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
172309,8461576750925894657,2016-08-07T00:00:00+03:00,2016-09-21T00:00:00+03:00,2800000.0,5,False,2,False,175.00,15.5,36.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Боровая улица, 18/1",5241676004767914301,16000.000000,75254.237288
232156,829522774249589760,2017-10-20T00:00:00+03:00,2018-02-07T00:00:00+03:00,2637000.0,3,False,2,False,64.34,NaN,NaN,NaN,0.0,1,2,"Россия, Санкт-Петербург, Рябовское шоссе, 139",6712797249654250862,40985.390115,57596.187175
34272,7410105106009286913,2016-03-10T00:00:00+03:00,2016-06-29T00:00:00+03:00,3000000.0,1,False,3,False,70.00,10.8,40.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Рябовское шоссе, 121к4",5362268880607991103,42857.142857,65789.473684
74271,6229460745561497601,2016-07-04T00:00:00+03:00,2016-09-02T00:00:00+03:00,1450000.0,6,False,1,False,31.00,10.0,15.00,NaN,2.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Московское шоссе, 270",4169477236027209826,46774.193548,73240.740741
172307,4332292566755429889,2016-08-07T00:00:00+03:00,2016-09-21T00:00:00+03:00,1200000.0,5,False,5,False,175.00,15.5,14.20,NaN,3.0,1,2,"Россия, Санкт-Петербург, Боровая улица, 18/1",5241676004767914301,6857.142857,75254.237288
249433,2922093125162048769,2018-02-09T00:00:00+03:00,2018-02-20T00:00:00+03:00,1250000.0,3,False,7,False,250.00,13.0,18.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Конторская улица, 16",6072468226709963670,5000.000000,5000.000000
232123,7460015048997193472,2017-09-18T00:00:00+03:00,2017-11-02T00:00:00+03:00,1432090.0,2,False,0,True,30.47,NaN,20.03,NaN,0.0,1,2,"Россия, Санкт-Петербург, Рябовское шоссе, 139",6712797249654250862,47000.000000,57596.187175
98450,244312844998338048,2017-09-08T00:00:00+03:00,2017-09-19T00:00:00+03:00,5950000.0,20,False,2,False,532.00,9.8,28.00,NaN,2.0,1,2,"Россия, Санкт-Петербург, Комендантский проспект, 51к1",2153540495534785186,11184.210526,109125.964010
192076,4112077486809808128,2017-07-10T00:00:00+03:00,2017-09-26T00:00:00+03:00,5770000.0,4,False,1,False,437.00,9.1,24.40,NaN,0.0,1,2,"Россия, Санкт-Петербург, Московский район, Авиационная улица, 15",6000386763082647198,13203.661327,137209.302326
23443,5452578904183591168,2017-10-12T00:00:00+03:00,2018-04-19T00:00:00+03:00,6000000.0,1,False,6,False,125.00,NaN,NaN,NaN,11.0,1,2,"Россия, Санкт-Петербург, улица Правды, 22",441998953379408884,48000.000000,136779.891875


In [91]:
# it seems that we should clean all apartments where median price is at least 2x higher and cheap 
outliers = sell_df_cleaned_spb[(sell_df_cleaned_spb.price_per_sq_m < 50000) 
                               & (sell_df_cleaned_spb.house_price_sqm_median/sell_df_cleaned_spb.price_per_sq_m >= 2)]

In [92]:
print("number of cheap outliers 2x cheaper: {}".format(len(outliers)))

number of cheap outliers 2x cheaper: 300


In [93]:
# clean outliers
sell_df_cleaned_spb = sell_df_cleaned_spb[~((sell_df_cleaned_spb.price_per_sq_m < 50000) 
                               & (sell_df_cleaned_spb.house_price_sqm_median/sell_df_cleaned_spb.price_per_sq_m >= 2))]

In [94]:
# there are cases where houses had just one offer and house median equals offer's price per square meter
# let's remove these outliers
sell_df_cleaned_spb = sell_df_cleaned_spb[~((sell_df_cleaned_spb.price_per_sq_m < 50000) 
                                          & (sell_df_cleaned_spb.price_per_sq_m == sell_df_cleaned_spb.house_price_sqm_median))]

3. Find houses with the most cheapest and most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers.

In [95]:
cheapest_house = sell_df_cleaned_spb[sell_df_cleaned_spb.house_price_sqm_median == sell_df_cleaned_spb.house_price_sqm_median.min()]
cheapest_house

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
78947,5709288895787653377,2016-08-24T00:00:00+03:00,2016-10-08T00:00:00+03:00,1250000.0,4,False,0,True,197.1,NaN,118.7,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Стойкости, 36к1",132296357019531359,6341.958397,7918.781726


In [96]:
#the most expensive house
expensive_house = sell_df_cleaned_spb[sell_df_cleaned_spb.house_price_sqm_median == sell_df_cleaned_spb.house_price_sqm_median.max()]
expensive_house

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
223172,1285755217853609883,2017-03-04T00:00:00+03:00,2017-04-02T00:00:00+03:00,115000000.0,1,False,3,False,151.7,30.0,60.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Санаторная аллея, 3",4057330062838252075,758075.148319,837837.837838
223173,1285755217853609886,2017-03-04T00:00:00+03:00,2017-04-02T00:00:00+03:00,120000000.0,2,False,4,False,153.4,25.0,110.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Санаторная аллея, 3",4057330062838252075,782268.578879,837837.837838
223175,1285755217853609861,2017-03-04T00:00:00+03:00,2017-04-02T00:00:00+03:00,137000000.0,1,False,3,False,201.2,50.0,99.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Санаторная аллея, 3",4057330062838252075,680914.512922,837837.837838
223176,1285755217853609887,2017-03-04T00:00:00+03:00,2017-04-02T00:00:00+03:00,218500000.0,2,False,5,False,244.9,20.0,110.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Санаторная аллея, 3",4057330062838252075,892200.898326,837837.837838
223177,1285755217853609885,2017-03-04T00:00:00+03:00,2017-04-02T00:00:00+03:00,220000000.0,2,False,4,False,257.6,25.0,110.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Санаторная аллея, 3",4057330062838252075,854037.267081,837837.837838
223178,1285755217853609888,2017-03-04T00:00:00+03:00,2017-04-02T00:00:00+03:00,220000000.0,3,False,4,False,225.3,23.0,110.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Санаторная аллея, 3",4057330062838252075,976475.810031,837837.837838


4.Find the most expensive and the most cheapest apartment in St. Petersburg after cleaning outliers.

In [97]:
cheapest_apart = sell_df_cleaned_spb[sell_df_cleaned_spb.last_price == sell_df_cleaned_spb.last_price.min()]
cheapest_apart

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
185561,5365283220161389569,2018-04-24T00:00:00+03:00,2018-04-27T00:00:00+03:00,1100000.0,1,False,0,True,33.0,NaN,NaN,NaN,3.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Первомайская улица, 5к3",6931516357826986190,33333.333333,62500.0


In [98]:
expensive_apart = sell_df_cleaned_spb[sell_df_cleaned_spb.last_price == sell_df_cleaned_spb.last_price.max()]
expensive_apart

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
237351,8310251211233707984,2018-03-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,605740032.0,2,False,5,False,443.0,10.0,248.0,0.0,NaN,1,2,"Россия, Санкт-Петербург, Южная дорога, 5",3312559599398519904,1.367359e+06,759301.442673


5. Calculate how many years does it take to cover all money spent on buying apartment by renting it. Find houses in which it's the most effective to invest in buying if you plan to rent and houses in which it will be the most ineffective.

In [99]:
# mergeing two dataframes on the building_id column
df = pd.merge(sell_df_cleaned_spb, rent_df_cleaned_spb, on='building_id', how='left')

In [100]:
# calculateing the number of years it takes to cover the cost of buying an apartment by renting it
df['years_to_cover_cost'] = df['last_price_x'] / (12 * df['last_price_y'])

In [101]:
# finding the house in which it's the most effective to invest in buying if you plan to rent
most_effective_houses = df[df['years_to_cover_cost'] == df['years_to_cover_cost'].min()]
most_effective_houses

,offer_id_x,first_day_exposition_x,last_day_exposition_x,last_price_x,floor_x,open_plan_x,rooms_x,studio_x,area_x,kitchen_area_x,living_area_x,agent_fee_x,renovation_x,offer_type_x,category_type_x,unified_address_x,building_id,price_per_sq_m_x,house_price_sqm_median_x,offer_id_y,first_day_exposition_y,last_day_exposition_y,last_price_y,floor_y,open_plan_y,rooms_y,studio_y,area_y,kitchen_area_y,living_area_y,agent_fee_y,renovation_y,offer_type_y,category_type_y,unified_address_y,price_per_sq_m_y,house_price_sqm_median_y,years_to_cover_cost
4723137,8311727660107294208,2016-04-15T00:00:00+03:00,2016-05-30T00:00:00+03:00,1268000.0,9,False,1,True,22.0,NaN,NaN,NaN,3.0,1,2,"Россия, Санкт-Петербург, переулок Гривцова, 6",6312852829589816564,57636.363636,99509.803922,1.949894e+18,2016-11-15T00:00:00+03:00,2016-11-23T00:00:00+03:00,250000.0,2.0,False,3.0,False,200.0,30.0,110.0,50.0,0.0,2.0,2.0,"Россия, Санкт-Петербург, переулок Гривцова, 6",1250.0,616.366366,0.422667


In [102]:
# finding the house in which it's the least effective to invest in buying if you plan to rent
least_effective_houses = df[df['years_to_cover_cost'] == df['years_to_cover_cost'].max()]
least_effective_houses

,offer_id_x,first_day_exposition_x,last_day_exposition_x,last_price_x,floor_x,open_plan_x,rooms_x,studio_x,area_x,kitchen_area_x,living_area_x,agent_fee_x,renovation_x,offer_type_x,category_type_x,unified_address_x,building_id,price_per_sq_m_x,house_price_sqm_median_x,offer_id_y,first_day_exposition_y,last_day_exposition_y,last_price_y,floor_y,open_plan_y,rooms_y,studio_y,area_y,kitchen_area_y,living_area_y,agent_fee_y,renovation_y,offer_type_y,category_type_y,unified_address_y,price_per_sq_m_y,house_price_sqm_median_y,years_to_cover_cost
3367375,5028522704340151610,2016-05-16T00:00:00+03:00,2017-01-09T00:00:00+03:00,602400000.0,9,False,8,False,525.8,23.5,308.6,NaN,6.0,1,2,"Россия, Санкт-Петербург, набережная реки Фонтанки, 76к2",8192868485509619740,1.145683e+06,500000.0,7.803040e+18,2016-09-26T00:00:00+03:00,2016-09-29T00:00:00+03:00,17000.0,5.0,False,1.0,False,32.0,NaN,NaN,0.0,11.0,2.0,2.0,"Россия, Санкт-Петербург, набережная реки Фонтанки, 76к2",531.25,1265.822785,2952.941176


In [103]:
s_df=sell_df_cleaned_spb[['building_id','unified_address','last_price']]
s_df.head(5)

,building_id,unified_address,last_price
0,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3676740.0
1,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3285640.0
2,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",4669280.0
3,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",4539560.0
4,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",4509750.0


In [104]:
#calculating mean price for every house
s_df = s_df.groupby(['building_id','unified_address'])['last_price'].mean().astype(int).reset_index()
s_df.head(5)

,building_id,unified_address,last_price
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",5198333
1,1785805724181110,"Россия, Санкт-Петербург, Рыбацкая улица, 6-8",8699000
2,3603962364551559,"Россия, Санкт-Петербург, Учебный переулок, 8к3",5942000
3,3603962364551561,"Россия, Санкт-Петербург, Учебный переулок, 8к1",5131250
4,3603962364553482,"Россия, Санкт-Петербург, Учебный переулок, 6к2",4750000


In [105]:
#renaming the solumns
s_df.columns = ['building_id', 'adress', 'mean_price_sell']

In [106]:
r_df=rent_df_cleaned_spb[['building_id','unified_address','last_price']]
r_df['price_per_year']=r_df['last_price']*12
r_df.head(5)

/tmp/ipykernel_468/4012395021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df['price_per_year']=r_df['last_price']*12


,building_id,unified_address,last_price,price_per_year
0,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",20000.0,240000.0
1,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",24000.0,288000.0
2,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",18000.0,216000.0
3,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",18000.0,216000.0
4,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",19000.0,228000.0


In [107]:
#calculating mean price for rent in every house
r_df = r_df.groupby(['building_id','unified_address'])['price_per_year'].mean().astype(int).reset_index()

In [108]:
#renaming the solumns
r_df.columns = ['building_id', 'adress', 'mean_price_per_year_rent']

In [109]:
merged_df = pd.merge(s_df, r_df, how='inner', on=['building_id', 'adress'])
merged_df

,building_id,adress,mean_price_sell,mean_price_per_year_rent
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",5198333,330000
1,3603962364551559,"Россия, Санкт-Петербург, Учебный переулок, 8к3",5942000,322000
2,3603962364551561,"Россия, Санкт-Петербург, Учебный переулок, 8к1",5131250,358000
3,3603962364553483,"Россия, Санкт-Петербург, Учебный переулок, 6к1",4610800,313000
4,13212466575073986,"Россия, Санкт-Петербург, Дунайский проспект, 34/16",6662580,404689
...,...,...,...,...
16457,9214053373585418118,"Россия, Санкт-Петербург, набережная реки Смоленки, 35к1",13527014,689765
16458,9214854470554944441,"Россия, Санкт-Петербург, Колпино, улица Анисимова, 2",4665882,228000
16459,9217640295586864077,"Россия, Санкт-Петербург, Гороховая улица, 17/56",8487500,684000
16460,9217640295588649554,"Россия, Санкт-Петербург, Гороховая улица, 35-37",9942857,410000


In [110]:
#calculating years to cover
merged_df['years_to_cover'] = (merged_df['mean_price_sell']/merged_df['mean_price_per_year_rent']).round(2)
merged_df

,building_id,adress,mean_price_sell,mean_price_per_year_rent,years_to_cover
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",5198333,330000,15.75
1,3603962364551559,"Россия, Санкт-Петербург, Учебный переулок, 8к3",5942000,322000,18.45
2,3603962364551561,"Россия, Санкт-Петербург, Учебный переулок, 8к1",5131250,358000,14.33
3,3603962364553483,"Россия, Санкт-Петербург, Учебный переулок, 6к1",4610800,313000,14.73
4,13212466575073986,"Россия, Санкт-Петербург, Дунайский проспект, 34/16",6662580,404689,16.46
...,...,...,...,...,...
16457,9214053373585418118,"Россия, Санкт-Петербург, набережная реки Смоленки, 35к1",13527014,689765,19.61
16458,9214854470554944441,"Россия, Санкт-Петербург, Колпино, улица Анисимова, 2",4665882,228000,20.46
16459,9217640295586864077,"Россия, Санкт-Петербург, Гороховая улица, 17/56",8487500,684000,12.41
16460,9217640295588649554,"Россия, Санкт-Петербург, Гороховая улица, 35-37",9942857,410000,24.25


In [111]:
merged_df.sort_values( by = 'years_to_cover', ascending = True)

,building_id,adress,mean_price_sell,mean_price_per_year_rent,years_to_cover
6937,3967985793187418353,"Россия, Санкт-Петербург, Боровая улица, 94",2230000,1800000,1.24
7939,4570380531975985556,"Россия, Санкт-Петербург, 11-я линия Васильевского острова, 18",1430000,960000,1.49
3399,1791270648516624245,"Россия, Санкт-Петербург, Малодетскосельский проспект, 7",1952000,1080000,1.81
4631,2540069387193465645,"Россия, Санкт-Петербург, Гродненский переулок, 4",1950000,1020000,1.91
3103,1627973416240229632,"Россия, Санкт-Петербург, 7-я Красноармейская улица, 7",4600000,2280000,2.02
...,...,...,...,...,...
6376,3567988545862627405,"Россия, Санкт-Петербург, Загородный проспект, 11",55000000,360000,152.78
3203,1703651731789727303,"Россия, Санкт-Петербург, Казанская улица, 27",33000000,216000,152.78
11956,6749677014473038546,"Россия, Санкт-Петербург, Алтайская улица, 4",29900000,180000,166.11
1197,536301776913345245,"Россия, Санкт-Петербург, Пушкин, Средняя улица, 32",58466666,300000,194.89
